# 🚀 SBAS — Sequential Batch Agent System
### Demo Notebook
**Patent pending 2025 · Lior Nataf & Matan Marudi**

This notebook demonstrates the core concepts of SBAS:
- State management for sequential agents
- Batch request grouping
- Cost savings tracking

## 1. State Manager Demo

In [1]:
import sys, subprocess
subprocess.run([sys.executable, '-m', 'pip', 'install', '-e', '..', '-q'], capture_output=True)

from sbas.state.memory import InMemoryStateManager

sm = InMemoryStateManager()
job_id = 'job-checkout-001'

state = {
    'step': 2,
    'messages': [
        {'role': 'system', 'content': 'You are a checkout analysis agent.'},
        {'role': 'user', 'content': 'Navigate to product page'},
        {'role': 'assistant', 'content': 'Product page loaded. Price: $29.99'},
    ],
    'context': {'product': 'Widget Pro', 'price': 29.99}
}

sm.save(job_id, state)
print(f'✓ State saved for: {job_id}')

restored = sm.load(job_id)
print(f'✓ Restored: step={restored["step"]}, messages={len(restored["messages"])}')

sm.update(job_id, {'step': 3, 'last_response': 'PayPal visible at checkout'})
print(f'✓ Updated to step: {sm.load(job_id)["step"]}')

✓ State saved for: job-checkout-001
✓ Restored: step=2, messages=3
✓ Updated to step: 3


## 2. Batch Queue Demo

In [2]:
from sbas.batch.queue import BatchQueue

queue = BatchQueue(max_size=100)

agents = [
    'fraud-detector-01',
    'checkout-agent-01',
    'checkout-agent-02',
    'terms-extractor-01',
    'ab-test-agent-01',
]

for agent in agents:
    queue.enqueue(
        job_id=f'job-{agent}',
        model='gpt-4o',
        messages=[{'role': 'user', 'content': f'Task for {agent}'}],
        kwargs={},
        client=None
    )
    print(f'  → Queued: {agent}')

print(f'\n✓ {len(queue._queue)} requests batched — 1 API call instead of {len(agents)}')
print(f'  Estimated savings: ~50% on all requests')

  → Queued: fraud-detector-01
  → Queued: checkout-agent-01
  → Queued: checkout-agent-02
  → Queued: terms-extractor-01
  → Queued: ab-test-agent-01

✓ 5 requests batched — 1 API call instead of 5
  Estimated savings: ~50% on all requests


## 3. Cost Tracker Demo

In [3]:
from sbas.cost.tracker import CostTracker

tracker = CostTracker()

class MockUsage:
    def __init__(self, i, o):
        self.prompt_tokens = i
        self.completion_tokens = o

class MockResponse:
    def __init__(self, model, i, o):
        self.model = model
        self.usage = MockUsage(i, o)

for i in range(3):
    tracker.record(f'sync-{i}', MockResponse('gpt-4o', 1200, 400), mode='sync')

for i in range(15):
    tracker.record(f'async-{i}', MockResponse('gpt-4o', 1200, 400), mode='async')

report = tracker.report()
print('💰 SBAS Cost Report')
print('=' * 40)
for k, v in report.items():
    label = k.replace('_', ' ').title()
    prefix = '$' if any(x in k for x in ['cost', 'saved', 'monthly']) else ''
    suffix = '%' if 'pct' in k else ''
    print(f'  {label:<28} {prefix}{v}{suffix}')

💰 SBAS Cost Report
  Total Calls                  18
  Sync Calls                   3
  Async Calls                  15
  Total Cost                   $0.126
  Cost If All Sync             $0.216
  Total Saved                  $0.09
  Savings Pct                  41.7%
  Projected Monthly            $2.7


## 4. Full End-to-End Flow (Patent Demo)

In [4]:
import uuid
from sbas.state.memory import InMemoryStateManager

print('🚀 Checkout Analysis Agent — Full Flow\n')

sm = InMemoryStateManager()
job_id = str(uuid.uuid4())
messages = [{'role': 'system', 'content': 'You are a checkout analysis agent.'}]

steps = [
    'Navigate to product page',
    'Add item to cart',
    'Proceed to checkout',
    'Analyze payment options shown',
    'Rate PayPal visibility 1-10',
]

for i, step in enumerate(steps):
    messages.append({'role': 'user', 'content': step})
    sm.save(job_id, {'step': i+1, 'messages': messages.copy()})
    print(f'  Step {i+1}: {step}')
    print(f'    → State saved | async LLM call submitted...')
    mock = f'[Async response for step {i+1}]'
    messages.append({'role': 'assistant', 'content': mock})
    sm.update(job_id, {'status': 'done', 'messages': messages.copy()})
    print(f'    → Response received, state reconstructed\n')

sm.delete(job_id)
print(f'✓ Complete! {len(steps)} steps | ~50% cost saved via async batch API')

🚀 Checkout Analysis Agent — Full Flow

  Step 1: Navigate to product page
    → State saved | async LLM call submitted...
    → Response received, state reconstructed

  Step 2: Add item to cart
    → State saved | async LLM call submitted...
    → Response received, state reconstructed

  Step 3: Proceed to checkout
    → State saved | async LLM call submitted...
    → Response received, state reconstructed

  Step 4: Analyze payment options shown
    → State saved | async LLM call submitted...
    → Response received, state reconstructed

  Step 5: Rate PayPal visibility 1-10
    → State saved | async LLM call submitted...
    → Response received, state reconstructed

✓ Complete! 5 steps | ~50% cost saved via async batch API


## Summary

| Component | Purpose | Impact |
|-----------|---------|--------|
| State Manager | Save/restore agent context | Enables async |
| Batch Queue | Group multiple requests | Reduces overhead |
| Async Routing | Use 50%-cheaper batch APIs | ~50% cost cut |
| Cost Tracker | Measure savings per job | Full visibility |

```python
# One line to start saving:
from sbas import SBAS
client = SBAS(OpenAI(), latency_budget='2h')
```